**How to plug Memory into a Chain?**

**LLMChain**

In [14]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
llm=ChatOpenAI(temperature=0.1)

memory=ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history"
)
template="""
    You are a helpful AI  talking to a human.
    {chat_history}
    Human:{question}
    You:
"""
chain=LLMChain(
    llm=llm,
    memory=memory,
    prompt=PromptTemplate.from_template(template),
    verbose=True,
)

chain.predict(question="My name is SeungWoo")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI  talking to a human.
    
    Human:My name is SeungWoo
    You:


> Finished chain.


'Hello SeungWoo! How can I assist you today?'

In [19]:
chain.predict(question="I live in Ilsan, Korea")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI  talking to a human.
    System: The human introduces themselves as SeungWoo.
AI: Hello SeungWoo! How can I assist you today?
Human: I live in Ilsan, Korea
AI: That's great to know! How can I assist you with living in Ilsan, Korea?
Human: What is my name?
AI: Your name is SeungWoo.
Human: I live in Ilsan, Korea
AI: That's great to know! How can I assist you with living in Ilsan, Korea?
    Human:I live in Ilsan, Korea
    You:


> Finished chain.


'AI: Yes, you mentioned that you live in Ilsan, Korea. Is there anything specific you would like assistance with related to living in Ilsan?'

In [16]:
chain.predict(question="What is my name?")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI  talking to a human.
    Human: My name is SeungWoo
AI: Hello SeungWoo! How can I assist you today?
Human: I live in Ilsan, Korea
AI: That's great to know! How can I assist you with living in Ilsan, Korea?
    Human:What is my name?
    You:


> Finished chain.


'Your name is SeungWoo.'

In [18]:
memory.load_memory_variables({})

{'chat_history': "System: The human introduces themselves as SeungWoo.\nAI: Hello SeungWoo! How can I assist you today?\nHuman: I live in Ilsan, Korea\nAI: That's great to know! How can I assist you with living in Ilsan, Korea?\nHuman: What is my name?\nAI: Your name is SeungWoo.\nHuman: I live in Ilsan, Korea\nAI: That's great to know! How can I assist you with living in Ilsan, Korea?"}

**ChatBasedMemory**

In [22]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate,ChatPromptTemplate,MessagesPlaceholder
llm=ChatOpenAI(temperature=0.1)

memory=ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
    return_messages=True,
)
prompt=ChatPromptTemplate.from_messages([
    ("system","You are a helpful AI talking to a human"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human","{question}"),

])
chain=LLMChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    verbose=True,
)
chain.predict(question="My name is SeungWoo")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
Human: My name is SeungWoo

> Finished chain.


'Nice to meet you, SeungWoo! How can I assist you today?'

In [23]:
chain.predict(question="I live in Ilsan, Korea")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
Human: My name is SeungWoo
AI: Nice to meet you, SeungWoo! How can I assist you today?
Human: I live in Ilsan, Korea

> Finished chain.


'Ilsan is a beautiful place! Is there anything specific you would like to know or talk about regarding Ilsan or Korea?'

In [24]:
chain.predict(question="What is my name?")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
Human: My name is SeungWoo
AI: Nice to meet you, SeungWoo! How can I assist you today?
Human: I live in Ilsan, Korea
AI: Ilsan is a beautiful place! Is there anything specific you would like to know or talk about regarding Ilsan or Korea?
Human: What is my name?

> Finished chain.


'Your name is SeungWoo.'

**LCEL Based Memory**

In [30]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import PromptTemplate,ChatPromptTemplate,MessagesPlaceholder
llm=ChatOpenAI(temperature=0.1)

memory=ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,
)
prompt=ChatPromptTemplate.from_messages([
    ("system","You are a helpful AI talking to a human"),
    MessagesPlaceholder(variable_name="history"),
    ("human","{question}"),
])
def load_memory(input):
    print(input)
    return memory.load_memory_variables({})["history"]
chain=RunnablePassthrough.assign(history=load_memory)|prompt|llm
def invoke_chain(question):
    result=chain.invoke({
        "question":"My name is SeungWoo",
    })
    memory.save_context(
        {"input":question},
        {"output":result.content},
    )
    print(result)


In [31]:
invoke_chain("My name is SeungWoo")

{'question': 'My name is SeungWoo'}
content='Nice to meet you, SeungWoo! How can I assist you today?'


In [32]:
invoke_chain("What is my name?")

{'question': 'My name is SeungWoo'}
content='Got it, SeungWoo! How can I assist you today?'
